In [1]:
import pandas as pd

# 파일 경로
file_path = "../../dataset/phishing_total.csv"

# CSV 파일 읽기
df = pd.read_csv(file_path)

# phishing_type이 '기관사칭형'인 데이터만 필터링
filtered_df = df[df["phishing_type"] == "기관사칭형"].reset_index(drop=True)

# 상위 5개 데이터 확인
filtered_df.head()

,file_name,phishing_type,speaker,text
0,phishing_686,기관사칭형,0,"안녕하세요, OOO님이신가요? 서울중앙지검 김검사입니다."
1,phishing_686,기관사칭형,1,"네, 맞는데요. 무슨 일이죠?"
2,phishing_686,기관사칭형,0,OOO님 명의로 범죄에 연루된 계좌가 발견되어 확인이 필요합니다.
3,phishing_686,기관사칭형,1,정말요? 어떻게 해야 하나요?
4,phishing_686,기관사칭형,0,우선 신분 확인을 위해 주민등록번호를 알려주셔야 합니다.


In [2]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27823 entries, 0 to 27822
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   file_name      27823 non-null  object
 1   phishing_type  27823 non-null  object
 2   speaker        27823 non-null  int64 
 3   text           27823 non-null  object
dtypes: int64(1), object(3)
memory usage: 869.6+ KB


In [12]:
# 1. file_name 기준으로 text 합치기
merged_text_series = filtered_df.groupby("file_name")["text"].apply("".join)

# 2. file_name 기준으로 phishing_type 추출 (하나만 가져오기)
phishing_type_series = filtered_df.groupby("file_name")["phishing_type"].first()

# 3. DataFrame으로 병합 및 컬럼 순서 지정
merged_df = pd.DataFrame(
    {
        "file_name": merged_text_series.index,
        "phishing_type": phishing_type_series.values,
        "text": merged_text_series.values,
    }
)

# 결과 확인
print(merged_df.head())
print(merged_df.info())

       file_name phishing_type  \
0  phishing_1000         기관사칭형   
1  phishing_1001         기관사칭형   
2  phishing_1002         기관사칭형   
3  phishing_1003         기관사칭형   
4  phishing_1004         기관사칭형   

                                                text  
0  안녕하세요, OOO 씨인가요? 우리은행 고객센터에서 연락드립니다.네, 저 OOO인데...  
1  안녕하세요, OOO님이신가요? 서울중앙경찰서 수사관 박준영입니다.네, 맞습니다. 무...  
2  여보세요, OOO님 맞으신가요? 서울지방검찰청 김혜민입니다.네, 제가 OOO입니다....  
3  안녕하세요, OOO님이신가요? 국민은행 보안부의 김선우입니다.네, 맞습니다. 무슨 ...  
4  여보세요, OOO님이시죠? 국세청 조사관 이지훈입니다.네, OOO입니다. 어떻게 도...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   file_name      861 non-null    object
 1   phishing_type  861 non-null    object
 2   text           861 non-null    object
dtypes: object(3)
memory usage: 20.3+ KB
None


In [6]:
total_chars = merged_text_series.str.len().sum()
print(f"총 글자 수: {total_chars}")

총 글자 수: 1022249


In [18]:
import pandas as pd
import requests
import time

# 1. 원본 CSV 로딩
df = pd.read_csv("../../dataset/phishing_total.csv")

# 2. '기관사칭형' 데이터 필터링
filtered_df = df[df["phishing_type"] == "기관사칭형"].reset_index(drop=True)
# 1. file_name 기준으로 text 합치기
merged_text_series = filtered_df.groupby("file_name")["text"].apply("".join)

# 2. file_name 기준으로 phishing_type 추출 (하나만 가져오기)
phishing_type_series = filtered_df.groupby("file_name")["phishing_type"].first()

# 3. DataFrame으로 병합 및 컬럼 순서 지정
merged_df = pd.DataFrame(
    {
        "file_name": merged_text_series.index,
        "phishing_type": phishing_type_series.values,
        "text": merged_text_series.values,
    }
)

# 3. Lingva Translate API 설정
BASE_URL = "https://lingva.ml/api/v1"
intermediate_langs = ["en", "ja", "zh"]  # 영어, 일본어, 중국어


# 4. 역번역 함수 정의
def back_translate(text, via_lang):
    try:
        # Step 1: 한국어 → 중간 언어
        res1 = requests.get(f"{BASE_URL}/ko/{via_lang}/{text}")
        res1.raise_for_status()
        translated = res1.json()["translation"]
        time.sleep(1)  # API 요청 제한 방지

        # Step 2: 중간 언어 → 한국어
        res2 = requests.get(f"{BASE_URL}/{via_lang}/ko/{translated}")
        res2.raise_for_status()
        back_translated = res2.json()["translation"]
        time.sleep(1)

        return back_translated
    except Exception as e:
        print(f"[ERROR lang={via_lang}] {e}")
        return None


# 5. 증강 데이터 생성
augmented_rows = []

for idx, row in merged_df.iterrows():
    original = row["text"]
    for lang in intermediate_langs:
        bt_text = back_translate(original, lang)
        if bt_text:
            augmented_rows.append(
                {
                    "file_name": row["file_name"],
                    "phishing_type": row["phishing_type"],
                    "text": bt_text,  # 증강된 텍스트
                    "original_text": original,
                    "via_language": lang,
                }
            )
        print(bt_text)

# 6. 결과 데이터프레임 생성
augmented_df = pd.DataFrame(augmented_rows)

안녕하세요, ooo입니다
안녕하세요, 이것은 oooo입니다
안녕하세요, 그렇습니다
안녕하세요, 당신은 ooo입니다
안녕하세요, oooo입니까?
안녕하세요, 당신은?
안녕하세요, 당신이 맞아요
안녕하세요, 그렇습니까?
안녕하세요, 맞습니까?
안녕하세요, 당신은 ooo입니다
안녕하세요, oooo입니까?
안녕하세요, 당신은?
안녕하세요, 당신은 ooo입니다
안녕하세요, 당신은 oooo입니다
안녕하세요, 당신은입니다
안녕하세요, 당신은 ooo입니다
안녕하세요, oooo입니까?
안녕하세요, 당신은?
안녕하세요, ooo가 맞습니다
안녕하세요, oooo가 맞습니다
안녕하세요, oo가 맞습니다
안녕하세요, 당신이 맞아요
안녕하세요, 그렇습니까?
안녕하세요, 맞습니까?
오, 안녕하세요


KeyboardInterrupt: 

In [ ]:
# 1. 원본 CSV 로딩
df = pd.read_csv("../../dataset/phishing_total.csv")

# 2. '기관사칭형' 데이터 필터링
filtered_df = df[df["phishing_type"] == "기관사칭형"].reset_index(drop=True)
# 1. file_name 기준으로 text 합치기
merged_text_series = filtered_df.groupby("file_name")["text"].apply("".join)

# 2. file_name 기준으로 phishing_type 추출 (하나만 가져오기)
phishing_type_series = filtered_df.groupby("file_name")["phishing_type"].first()

# 3. DataFrame으로 병합 및 컬럼 순서 지정
merged_df = pd.DataFrame(
    {
        "file_name": merged_text_series.index,
        "phishing_type": phishing_type_series.values,
        "text": merged_text_series.values,
    }
)

NameError: name 'augmented_df' is not defined

In [ ]:
# --- 7. 최종 데이터셋을 CSV 파일로 저장 ---
output_csv_path = "../../dataset/역번역데이터셋.csv"
try:
    merged_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(f"\n▶ 최종 데이터셋이 ' {output_csv_path}' 파일로 성공적으로 저장되었습니다.")
except Exception as e:
    print(f"\n오류: 최종 데이터셋 저장 중 문제가 발생했습니다: {e}")

In [ ]:
# 7. 저장 또는 기존 데이터와 결합
augmented_df.to_csv("augmented_phishing_lingva.csv", index=False)

# (선택) 기존과 결합
# combined_df = pd.concat([filtered_df, augmented_df[filtered_df.columns]], ignore_index=True)

# 기관사칭형 데이터셋 증강 

In [73]:
df = pd.read_csv("../../dataset/grouped_기관사칭형.csv")
df

,file_name,phishing_type,speaker,text
0,phishing_686,기관사칭형,0,"안녕하세요, OOO님이신가요? 서울중앙지검 김검사입니다."
1,phishing_686,기관사칭형,1,"네, 맞는데요. 무슨 일이죠?"
2,phishing_686,기관사칭형,0,OOO님 명의로 범죄에 연루된 계좌가 발견되어 확인이 필요합니다.
3,phishing_686,기관사칭형,1,정말요? 어떻게 해야 하나요?
4,phishing_686,기관사칭형,0,우선 신분 확인을 위해 주민등록번호를 알려주셔야 합니다.
...,...,...,...,...
27818,phishing_1951,기관사칭형,1,네 알겠습니다
27819,phishing_1951,기관사칭형,0,네 여보세요
27820,phishing_1951,기관사칭형,1,네 여보세요
27821,phishing_1951,기관사칭형,0,예 예 녹취를 한번 끊었다 갈게요


In [74]:
# 필수 라이브러리 import
import pandas as pd
import time
import os
from dotenv import load_dotenv
# from tqdm.notebook import tqdm
from tqdm import tqdm

# OpenAI 1.0.0 이상용 클라이언트 import
from openai import OpenAI

# ✅ .env에서 API 키 불러오기
print("🔐 .env에서 OpenAI API 키 로딩 중...")
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("✅ OpenAI API 키 로딩 완료")
else:
    raise ValueError("❌ OpenAI API 키를 불러올 수 없습니다. .env 파일을 확인해주세요.")

# ✅ OpenAI 클라이언트 생성
client = OpenAI(api_key=api_key)

🔐 .env에서 OpenAI API 키 로딩 중...
✅ OpenAI API 키 로딩 완료


# 오픈 AI에게 요청하는 함수

In [75]:
def gpt_back_translate_zh(text):
    try:
        print(f"🔁 역번역 시작 (원문 일부): {text[:30]}...")
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "너는 한국 보이스피싱 탐지 시스템 프로젝트의 데이터 증강 전문가야. "
                        "사용자가 주는 한국어 문장은 보이스피싱 사례 텍스트야. "
                        "너의 임무는 이 문장을 먼저 중국어로 자연스럽게 번역한 뒤, 다시 한국어로 자연스럽고 다양하게 재구성해주는 거야. "
                        "단, 원래 문장의 의미와 맥락은 유지하되 표현을 바꾸고, 자연스럽고 실제 통화처럼 들리도록 만들어야 해. "
                        "**최종 출력은 오직 자연스럽고 일상적인 한국어 문장 하나만 포함해야 해.** "
                        "출력에 중국어 단어가 섞이거나 설명이 포함되면 실패야."
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        f"다음 문장을 중국어로 번역한 뒤, 다시 자연스럽고 실제 대화처럼 들리게 한국어로 재번역해줘. "
                        f"출력에는 반드시 한국어 문장만 포함돼야 해. 중국어 단어, 설명, 번역 과정은 절대 포함하지 마:\n\n'{text}'"
                    ),
                },
            ],
            temperature=1.0,
            max_tokens=500,
        )
        result = response.choices[0].message.content.strip()
        print(f"✅ 역번역 완료 (결과 일부): {result[:30]}...")
        return result

    except Exception as e:
        print(f"❌ 역번역 중 오류 발생: {e}")
        return None

# 실행중 오류시 복구하고 이어서 증강하는 코드

# 백업기능 실행코드 

In [77]:
# import pandas as pd
import os
import time
import datetime
import traceback
from tqdm.notebook import tqdm

# ✅ tqdm 게이지: 노트북 or 콘솔 자동 감지
# try:
#     from tqdm.notebook import tqdm
# except ImportError:
#     from tqdm import tqdm

# ✅ 설정값
lang_code = "zh"  # 역번역 언어
phishing_type = "기관사칭형"
backup_dir = "../../dataset/backup_augmented_data"
os.makedirs(backup_dir, exist_ok=True)


# ✅ 백업 파일명 생성 함수
def generate_filename(step=""):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    return os.path.join(
        backup_dir, f"{phishing_type}_{lang_code}_backup_{step}_{timestamp}.csv"
    )


# ✅ 백업 저장 함수
def save_backup(rows, step="unknown"):
    try:
        temp_df = pd.DataFrame(rows)
        path = generate_filename(step)
        temp_df.to_csv(path, index=False, encoding="utf-8-sig")
        print(f"✅ [백업 저장] step={step} → {path}")
    except Exception as e:
        print(f"❌ 백업 실패: {e}")


# ✅ 결과 저장용 리스트
augmented_rows = []

# ✅ 역번역 루프 시작
for idx, row in tqdm(
    df.iterrows(), total=len(df), desc=f"{phishing_type} 역번역 중 ({lang_code})"
):
    try:
        translated_text = gpt_back_translate_zh(row["text"])

        if translated_text:
            augmented_rows.append(
                {
                    "file_name": f"{row['file_name']}_{lang_code}",
                    "phishing_type": row["phishing_type"],
                    "speaker": row["speaker"],
                    "text": translated_text,
                }
            )

        # ✅ 주기적 백업 (50개마다)
        if idx % 50 == 0 and idx != 0:
            save_backup(augmented_rows, step=f"{idx}rows")

        time.sleep(0.3)  # 속도 조절

    except Exception as e:
        print(f"⚠️ 오류 발생 @ row {idx}: {e}")
        traceback.print_exc()
        save_backup(augmented_rows, step=f"error_{idx}")
        continue

# ✅ 최종 저장
final_df = pd.DataFrame(augmented_rows)
final_filename = f"{phishing_type}_{lang_code}_dataset.csv"
final_df.to_csv(final_filename, index=False, encoding="utf-8-sig")
print(f"🎉 완료! 최종 저장 → {final_filename} ({len(final_df)} rows)")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html